<a href="https://colab.research.google.com/github/touhu27344/spaceship-titanic/blob/main/spaceship_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  **Kaggle start up**

In [1]:
from google.colab import drive
#ドライブのマウント
drive.mount('/content/drive')

#kaggle.json をcolab上にアップロード
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

#kaggleAPIのインストール
!pip install kaggle

#kaggle.json に権限付与
!chmod 600 /root/.kaggle/kaggle.json

#ライブラリのパスを追加
import sys
LIBLARY = "/content/drive/MyDrive/Colab\ Notebooks/Kaggle/libs"
sys.path.append(LIBLARY)

Mounted at /content/drive




---


kaggle terminal

### **kaggleAPI コマンド**


1. ``` !kaggle competitions list ``` :開催中のコンペ一覧を表示
2. ``` !kaggle competitions files -c [コンペ名] ``` :指定のコンペの使用するデータを表示
3. ``` !kaggle competitions download -c [コンペ名] ``` :指定のコンペのデータをカレントにzipでダウンロード
4. ``` !kaggle competitions submit -c [コンペ名] -f [ファイル名] -m [コメント] ``` :コンペに予測値をコメント付きでsubmit
5. ``` !kaggle competitions submissions -c [コンペ名] ``` :コンペに投降した自分のsubmit記録一覧を表示

参考：https://qiita.com/takuma-1234/items/c46007d2b83bb9b569fa



In [2]:
!kaggle competitions list

ref                                                                                deadline             category                reward  teamCount  userHasEntered  
---------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2      2025-04-01 23:59:00  Featured         2,117,152 Usd        816           False  
https://www.kaggle.com/competitions/konwinski-prize                                2025-03-12 23:59:00  Featured         1,225,000 Usd         93           False  
https://www.kaggle.com/competitions/gemma-language-tuning                          2025-01-15 00:59:00  Analytics          150,000 Usd          0           False  
https://www.kaggle.com/competitions/jane-street-real-time-market-data-forecasting  2025-01-13 23:59:00  Featured           120,000 Usd       2933           False  
https://www.kagg

In [3]:
!kaggle competitions download -c spaceship-titanic
compeName = "spaceship-titanic"

100% 299k/299k [00:00<00:00, 681kB/s]
100% 299k/299k [00:00<00:00, 680kB/s]


ダウンロードしたzipファイルの解凍

In [4]:
#zipファイルの解凍

#使用するライブラリをimportする
import shutil
import os

#カレントディレクトリから、「.zip」を含むものだけ取り出す。
zip_file = [x for x in os.listdir() if '.zip' in x]
zip_file

#zipファイルを解答する処理
shutil.unpack_archive(zip_file[0], './drive/MyDrive/Colab Notebooks/'+ compeName +'/input')

!pip install [モジュール名] -t /content/drive/MyDrive/Colab\ Notebooks/Kaggle/libs

In [11]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('Not found GPU')
else:
    print('Found GPU at: {}'.format(device_name))

Not found GPU


===============================================

In [96]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import gc
import warnings
warnings.filterwarnings('ignore')

In [97]:
%%time
# Load datasets using context managers
def load_datasets():
    try:
        test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/'+ compeName +'/input/test.csv')
        train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/'+ compeName +'/input/train.csv')

        print("Datasets loaded successfully.")
        return test_data, train_data

    except Exception as e:
        print(f"Error loading datasets: {e}")
        return None


test_data , train_data = load_datasets()

Datasets loaded successfully.
CPU times: user 50.1 ms, sys: 1.99 ms, total: 52.1 ms
Wall time: 67 ms


In [98]:
def display_dataset_info(dataset, name):
    print("-----------------------------------------------------------------")
    print(f"{name} DataFrame Shape: Rows = {dataset.shape[0]}, Columns = {dataset.shape[1]}")

    # Numerical and categorical columns information
    num_cols = dataset.select_dtypes(include='number')
    cat_cols = dataset.select_dtypes(exclude='number')
    print(f"{name} DataFrame has {len(num_cols.columns)} numeric columns and {len(cat_cols.columns)} categorical columns.")
    print(print(f"numeric column.:") )
    for num_col in num_cols:
        print(f"[{num_col}]")
    print(print(f"categorical column.:") )
    for cat_col in cat_cols:
        print(f"[{cat_col}]")

    # Missing values information
    total_missing = dataset.isnull().sum().sum()
    if total_missing > 0:
        missing_perc = (total_missing / (dataset.shape[0] * dataset.shape[1])) * 100
        print(f"There are a total of {total_missing} missing values in the {name} DataFrame ({missing_perc:.2f}% of all values).")
        print("Missing values per column:")
        print(dataset.isnull().sum().sort_values(ascending=False).head(10))
    else:
        print(f"There are no missing values in the {name} DataFrame.")

    # Duplicate rows information
    total_duplicates = dataset.duplicated().sum()
    if total_duplicates > 0:
        print(f"There are {total_duplicates} duplicate rows in the {name} DataFrame.")
    else:
        print(f"There are no duplicate rows in the {name} DataFrame.")

    # Check for column data types
    print("\nColumn data types:")
    print(dataset.dtypes.value_counts())

    print("-----------------------------------------------------------------")


datasets = [test_data, train_data]
names = [ "test_data","train_data"]
for i in range(len(datasets)):
    display_dataset_info(datasets[i], names[i])

-----------------------------------------------------------------
test_data DataFrame Shape: Rows = 4277, Columns = 13
test_data DataFrame has 6 numeric columns and 7 categorical columns.
numeric column.:
None
[Age]
[RoomService]
[FoodCourt]
[ShoppingMall]
[Spa]
[VRDeck]
categorical column.:
None
[PassengerId]
[HomePlanet]
[CryoSleep]
[Cabin]
[Destination]
[VIP]
[Name]
There are a total of 1117 missing values in the test_data DataFrame (2.01% of all values).
Missing values per column:
FoodCourt       106
Spa             101
Cabin           100
ShoppingMall     98
Name             94
CryoSleep        93
VIP              93
Destination      92
Age              91
HomePlanet       87
dtype: int64
There are no duplicate rows in the test_data DataFrame.

Column data types:
object     7
float64    6
Name: count, dtype: int64
-----------------------------------------------------------------
-----------------------------------------------------------------
train_data DataFrame Shape: Rows = 86

In [99]:
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [100]:
# Function to reduce memory usage
def reduce_memory(df,name):
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage before optimization: {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            if str(col_type).startswith("int"):
                df[col] = pd.to_numeric(df[col], downcast="integer")
            elif str(col_type).startswith("float"):
                df[col] = pd.to_numeric(df[col], downcast="float")
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print(f"{ name } Memory usage after optimization: {end_mem:.2f} MB")
    print(f"Reduced by {(100 * (start_mem - end_mem) / start_mem):.1f}%")
    print("-----------------------------")
    return df

for i in range(len(datasets)):
    reduce_memory(datasets[i], names[i])

Memory usage before optimization: 0.42 MB
test_data Memory usage after optimization: 0.61 MB
Reduced by -43.0%
-----------------------------
Memory usage before optimization: 0.87 MB
train_data Memory usage after optimization: 1.23 MB
Reduced by -41.0%
-----------------------------


In [101]:
# Feature engineering
train_data[["Deck","Num","Side"]] = train_data["Cabin"].str.split('/', expand = True)
train_data.drop(columns=["PassengerId","Cabin"],inplace = True)

In [102]:
train_data[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = train_data[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(value=0)
train_data["Transported"] = train_data["Transported"].astype(int)
train_data["VIP"] = train_data["VIP"].fillna(False).astype(int)
#train_data["CryoSleep"] = train_data["CryoSleep"].fillna(False).astype(int)

In [103]:
train_data

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side
0,Europa,False,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,1,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0,A,98,P
8689,Earth,True,PSO J318.5-22,18.0,0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0,G,1499,S
8690,Earth,False,TRAPPIST-1e,26.0,0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1,G,1500,S
8691,Europa,False,55 Cancri e,32.0,0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0,E,608,S


In [104]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [107]:
train_data0 = train_data.sample(frac=0.6, random_state= 42)[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','CryoSleep']].dropna(subset=['CryoSleep'])

X = train_data0.drop(columns=['CryoSleep'],axis=1)
y = train_data0['CryoSleep'].astype(int)

model1 = RandomForestRegressor(n_estimators=125, random_state=42)
model2 = LinearRegression()
model3 = GradientBoostingRegressor(n_estimators=125, learning_rate = 0.05, max_depth=4, random_state=42)

CryoSleep_model = VotingRegressor(estimators=[('rf', model1),('lr',model2),('gb', model3)])
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit pipeline
CryoSleep_model.fit(X_train, y_train)

y_pred = CryoSleep_model.predict(X_test)
y_pred = np.where(y_pred >= 0.5, True, False)
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.93


In [80]:
# 1. Identify rows with missing 'CryoSleep' values
missing_cryosleep_rows = train_data[train_data['CryoSleep'].isnull()]

# 2. Prepare features for prediction
X_missing = missing_cryosleep_rows[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]

# 3. Predict 'CryoSleep' values using the model
cryosleep_predictions = CryoSleep_model.predict(X_missing)

# 4. Impute the predictions into the original DataFrame
train_data.loc[train_data['CryoSleep'].isnull(), 'CryoSleep'] = cryosleep_predictions >= 0.5

train_data["CryoSleep"] = train_data["CryoSleep"].astype(int)


In [81]:
%%time

train_data1 = train_data.sample(frac=0.6, random_state=42)
# Prepare data
X = train_data1.drop(columns=["Transported"], axis=1)
y = train_data1['Transported']

numerical_cols = X.select_dtypes(include=["float64", "int64"]).columns
categorical_cols = X.select_dtypes(include=["object"]).columns

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ]
)

# Models
model1 = RandomForestRegressor(n_estimators=125, random_state=42)
model2 = LinearRegression()
model3 = GradientBoostingRegressor(n_estimators=125, learning_rate = 0.05, max_depth=4, random_state=42)

ensemble_model = VotingRegressor(estimators=[('rf', model1),('lr',model2),('gb', model3)])

# Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('ensemble', ensemble_model)
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit pipeline
pipeline.fit(X_train, y_train)



CPU times: user 2min 1s, sys: 234 ms, total: 2min 1s
Wall time: 2min 1s


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  Index(['CryoSleep', 'VIP'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  Index(['Deck', 'Num', 'Side'], dtype='object'))])),
                ('ensemble',
                 VotingRegressor(estimators=[('rf',
                                              RandomForestRegressor(n_estimators=125,
                                                                    random_state=42)),
                                             ('lr', LinearRegression()),
                                             ('gb',
                                              GradientBoostingRegressor(learning_rate=0.05,
                                                                        max_depth=4,
                                                                        n_estimators=125,
                                                                        random_state=42))]))])

In [89]:
# Predictions and evaluation
y_pred = pipeline.predict(X_test)
y_pred = np.where(y_pred >= 0.5, True, False)
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.72


In [90]:
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/spaceship-titanic/input/test.csv')

# Feature engineering
test_data[["Deck","Num","Side"]] = test_data["Cabin"].str.split('/', expand = True)
passengerId = test_data["PassengerId"]
test_data.drop(columns=["PassengerId","Cabin"],inplace = True)


test_data[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = test_data[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(value=0)
test_data["VIP"] = test_data["VIP"].fillna(False).astype(int)
test_data["CryoSleep"] = test_data["CryoSleep"].fillna(False).astype(int)

categorical_cols = test_data.select_dtypes(include=["object"]).columns
oneHotEncoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
oneHotEncoder.fit(test_data)

# OneHotEncode
categorical_cols = test_data.select_dtypes(include=["object"]).columns
oneHotEncoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
oneHotEncoder.fit(test_data)

# 数値特徴量用の補完器を作成
from sklearn.impute import SimpleImputer
num_imputer = SimpleImputer(strategy='mean') # または 'median'

# test_data から数値列を取得
num_cols = test_data.select_dtypes(include=np.number).columns

# 数値列の欠損値を補完
test_data[num_cols] = num_imputer.fit_transform(test_data[num_cols])



In [91]:
# pipeline はトレーニング済みのモデルと仮定します
test_predictions = pipeline.predict(test_data)

In [63]:
test_predictions

array([0.73896464, 0.29743569, 0.75078476, ..., 0.3848661 , 0.33161793,
       0.70555487])

In [92]:
submission_df = pd.DataFrame({
    'PassengerId': passengerId,
    'Transported': np.where(test_predictions >= 0.5, True, False)
})

In [93]:
submission_df.to_csv('/content/drive/MyDrive/Colab Notebooks/spaceship-titanic/output/submit.csv', index=False)

In [94]:
!kaggle competitions submit -c spaceship-titanic -f '/content/drive/MyDrive/Colab Notebooks/spaceship-titanic/output/submit.csv' -m seconds

100% 56.8k/56.8k [00:01<00:00, 39.6kB/s]
Successfully submitted to Spaceship Titanic

In [95]:
!kaggle competitions submissions -c spaceship-titanic

fileName    date                 description  status    publicScore  privateScore  
----------  -------------------  -----------  --------  -----------  ------------  
submit.csv  2024-12-23 14:02:14  seconds      complete  0.71405                    
submit.csv  2024-12-23 13:55:17  seconds      complete  0.57774                    
submit.csv  2024-12-23 13:30:59  seconds      complete  0.71662                    
submit.csv  2024-12-20 13:09:15  seconds      complete  0.71662                    
submit.csv  2024-12-20 12:59:33  seconds      complete  0.72270                    
submit.csv  2024-12-20 12:20:19  seconds      complete  0.58826                    
submit.csv  2024-12-20 12:07:23  seconds      complete  0.50736                    
submit.csv  2024-12-20 12:05:10  seconds      complete  0.58685                    
submit.csv  2024-12-20 11:40:08  seconds      complete  0.58662                    
submit.csv  2024-12-20 11:37:40  seconds      error                         